# Forschungsfrage 4
Im Folgendem wird die Forschungsfrage 4 behandelt.

	Welche sind die führenden User bzw. Experten und wie tragen diese zu den diskutierten Themen bei? 

# Importieren der Module
In der folgenden Zelle werden die benötigten Module, wie zum Beispiel Spacy und scikit-learn importiert.

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
import spacy 
from spacy.matcher import Matcher, PhraseMatcher
import pandas as pd
import re
from collections import Counter
import sklearn
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.transform import jitter
from bokeh.io import curdoc
from bokeh.models import LabelSet
import csv
import math
import networkx as nx
import nx_altair as nxa
import altair as alt
from networkx.drawing.nx_agraph import graphviz_layout

Die Konfiguration der unteren Zelle dient der Auswahl der Datensätze. Um alle Datensätze zu erfassen ist start_post = 0 und end_post = 42637 zu setzen

In [ ]:
# configuration
n_tag_posts = 500
n_answer_posts = 42637

# Datensätze laden
Als nächstes müssen die Datensätze aus der exportierten CSV-Datei geladen werden.

In [2]:
# Load csv data
filepath = os.path.join(os.getcwd(), 'QueryResults_sample_42636_14_05_21.csv')
stack_posts = pd.read_csv(filepath, sep = ",")

print("loaded csv data")

loaded csv data


Im folgendem werden die geladenen Datensätze ausgegeben

In [4]:
stack_posts

,QuestionUserId,QuestionUserReputation,QuestionUserDN,Tags,QuestionId,QuestionScore,title,QuestionBody,QuestionDate,AcceptedAnswer,AnswerUserId,AnswerUserReputation,AnswerUserDN,AnswerScore,AnswerId,AnswerBody,AnswerDate
0,3625340,33,user3625340,<image-processing><machine-learning><svm><feat...,27729199,0,How to find Relevent Features for Comparing Di...,<p>Currently we are doing a project on diagram...,2015-01-01 08:03:13,27733517.0,1056563,45925,StephenBoesch,0,27733517,<p>In regard solely to the difference in scale...,2015-01-01 18:39:02
1,4409773,788,Avis,<java><machine-learning><svm><encog>,27729238,1,SVM using Encog in Java for beginners,<p>I am beginner in SVM. Could someone please ...,2015-01-01 08:10:29,27808712.0,173355,3162,JeffHeaton,1,27808712,<p>In Encog SVM is just a classification or re...,2015-01-06 22:58:03
2,4408281,715,datavinci,<python-2.7><machine-learning>,27730775,1,Why does not the following code snippet run su...,<p>I was reading Programming Collective Intell...,2015-01-01 12:22:49,27730829.0,367273,436785,NPE,1,27730829,<blockquote>\n <p>NameError: global name 'lin...,2015-01-01 12:32:19
3,3512217,119,Shlomi,<machine-learning><svm>,27730870,-1,division of two proper kernels,"<p>Let <img src=""https://i.stack.imgur.com/Z1G...",2015-01-01 12:37:59,NaN,1060350,70610,Has QUIT--Anony-Mousse,0,27742921,"<p>K2(x,z) can be 0.</p>\n\n<p>Then this value...",2015-01-02 13:22:27
4,4405757,14440,user7,<machine-learning><classification><weka><libsv...,27732503,0,One class SVM to detect outliers,<p>My problem is</p>\n\n<blockquote>\n <p>I w...,2015-01-01 16:26:06,NaN,1060350,70610,Has QUIT--Anony-Mousse,5,27739848,<p>Your data is not formatted appropriately fo...,2015-01-02 09:20:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42631,14926602,21,Waqar Kaleem Khan,<tensorflow><machine-learning><keras><deep-lea...,67441958,0,LSTM input layer shape in Keras using function...,<p>I am trying to implement LSTMs on drug data...,2021-05-07 21:38:47,NaN,5927701,2972,data_person,1,67442769,<p>Try adding an <code>Embedding Layer</code> ...,2021-05-07 23:36:34
42632,3973175,4261,con,<python><python-3.x><machine-learning><shap>,67443411,2,How to get feature names of shap_values from T...,"<p>I am doing a shap tutorial, and attempting ...",2021-05-08 01:56:51,67444552.0,3954379,5202,Lucas,2,67444552,"<p>The features are indeed in the same order, ...",2021-05-08 06:02:39
42633,15847767,1,Pearl,<python><machine-learning><keras><neural-network>,67443744,-1,"What does hidden_layer = layers.Dense(100, act...","<p>I saw the following in <a href=""https://www...",2021-05-08 03:14:14,NaN,15166370,45,Vibhav Surve,0,67443785,<p>See input layer is nothing but how many neu...,2021-05-08 03:22:03
42634,14551464,61,Onkar Chougule,<machine-learning><deep-learning><pytorch><con...,67449276,0,Initalize using previous .pth and train for fu...,<p>How do I initialize a UNet model from its p...,2021-05-08 15:36:01,NaN,14551464,61,Onkar Chougule,0,67451326,<pre><code>state= torch.load(&quot;/content/mo...,2021-05-08 19:21:01


# Datenbereinigung
Die nächste Zelle definiert eine Funktion zum Bereinigen der Tags, um spitze Klammern zu entfernen.

In [3]:
# cleaning functions

# clean all angle brackets from string
def clean_tags(tags):    
    tag_list = []
    # clean tags from '>' and '<' occurences
    tags = re.sub('><', ' ', tags) 
    tags = re.sub('<|>', '', tags)
    # add single tag of tags and add it to lists and sets
    for tag in tags.split():
        if tag != 'machine-learning':
            tag_list.append(tag)
    return tag_list



In [4]:
# drop all duplicates in posts
df_w_dupes = pd.DataFrame(stack_posts[0:n_answer_posts])
df_wo_dupes = df_w_dupes.drop_duplicates(["QuestionId"])
df_sorted_wo_dupes = df_wo_dupes.sort_values(by=["QuestionId"])

# Useranalyse
Im Folgendem werden die Statistiken der User erstellt. Hierzu gehört unteranderem das Erstellen des User-Objekts, die Berechnung der Reputation und Speicherung der jeweiligen Tags.

In [5]:
# extact user stats (reputation in question/answers, question-/answers counts, amount of accepted answers etc.)
users = dict()

def create_user_stats(id, display_name):
    # add user if userid does not exist
    if id not in users:
        users[id] = dict()    
        users[id]["display_name"] = display_name
        
# customize stats which every user has
def customize_user_base_stats(id, score, post_type):    
    users[id][post_type + "_reputation"] = users[id].get(post_type + "_reputation", 0) + score * 10   
    users[id][post_type + '_count'] = users[id].get(post_type + '_count', 0) + 1


# iterate over dataframe tuples
for i, post in enumerate(df_w_dupes.itertuples()):  
    # create DataFrame from pandas.core.frame so column names can be used instead of indexes
    post = pd.DataFrame(post).transpose().drop(0, axis=1)
    post.columns = stack_posts.columns
    
    # customize question user stats if question is not a duplicate of previous question
    if post["QuestionId"][0] != df_w_dupes.iloc[i - 1]["QuestionId"]:
        question_user_id = post["QuestionUserId"][0]
        create_user_stats(question_user_id, post["QuestionUserDN"][0])
        
        # customize question user stats        
        customize_user_base_stats(question_user_id,post["QuestionScore"][0], "question")
        cleaned_tags = clean_tags(post["Tags"][0])
        users[question_user_id]["question_tags"] = set(users[question_user_id].get("question_tags", set()).union(cleaned_tags))
                    
                
    # create answer user stats
    answer_user_id = post["AnswerUserId"][0]
    create_user_stats(answer_user_id, post["AnswerUserDN"][0])
        
    customize_user_base_stats(answer_user_id,post["AnswerScore"][0], "answer")
    users[answer_user_id]["answer_tags"] = set(users[answer_user_id].get("answer_tags", set()).union(cleaned_tags))
    if post["AcceptedAnswer"][0] == post["AnswerId"][0]:
        users[answer_user_id]["accepted_answer_count"] = users[answer_user_id].get("accepted_answer_count", 0) + 1
        users[answer_user_id]["answer_reputation"] = users[answer_user_id].get("answer_reputation", 0) + 15

Der entstandene Dataframe wird nun transponiert und nach der Reputation der Antworten sortiert ausgegeben.

In [6]:
user_df = pd.DataFrame(users).transpose()
sorted_user_df = user_df.sort_values(by=['answer_reputation'], ascending=False)
#sorted_user_df = sorted_user_df[0:100]

In [8]:
sorted_user_df[0:20]

,display_name,question_reputation,question_count,question_tags,answer_reputation,answer_count,answer_tags,accepted_answer_count
4685471,desertnaut,10,1,"{shap, scikit-learn}",31680,579,"{hyperparameters, random-forest, ios, kaggle, ...",458
2658050,lejlot,NaN,NaN,NaN,25015,618,"{documentation, twitter, random-forest, time-s...",363
5974433,Marcin Możejko,70,3,"{embedding, neural-network, nlp, grid-search, ...",22335,238,"{hyperparameters, google-cloud-platform, plot,...",159
712995,Maxim,40,1,"{neural-network, python, deep-learning, tensor...",20950,378,"{hyperparameters, doc2vec, seq2seq, machine-le...",244
1714410,Shai,1020,9,"{neural-network, python, gradient-descent, lin...",16625,309,"{tensor, matplotlib, reduction, performance, d...",195
2099607,today,NaN,NaN,NaN,14080,318,"{signal-processing, hyperparameters, predict, ...",250
3374996,Vivek Kumar,NaN,NaN,NaN,9175,182,"{hyperparameters, random-forest, gaussian, dat...",129
5545260,dga,NaN,NaN,NaN,8445,14,"{c++, neural-network, image-processing, python...",9
2097240,Daniel Möller,NaN,NaN,NaN,8265,186,"{tensor, time-series, optimization, recurrent-...",105
562769,Martin Thoma,490,6,"{c++, image-processing, amazon-sagemaker, tens...",7830,92,"{handwriting-recognition, dbscan, data-science...",32


Anschließend werden nur die Posts der führenden 20 Experten für weitere Analysen ausgewählt.

In [11]:
# select posts from top users
top_answer_users = sorted_user_df[0:20]
top_users_posts = stack_posts[stack_posts["AnswerUserId"].isin(top_answer_users.index)]

In [7]:
# clean answers
regex_pattern = '(<(pre|code|blockquote|a|strike)(.|\n)*?\/(pre|code|blockquote|a|strike)>)*?|<(p|b|br|br(.|\n)*?\/|sub|sup|em|strong|hr|s|i|ol|ul|li|code)*?>|<\/(p|b|br|sub|sup|em|strong|s|i|ol|ul|li|div|pre|blockquote|a|code)>|<h(.|\n)*?>(.|\n)*?<\/h(.|\n)*?>*?|(<(img|div|ol|ul|li)(.|\n)*?\/*?>)|\n'
def clean_bodys(text):
    text = re.sub(regex_pattern, '', text, flags=re.I)
    text = re.sub('\(|\)', ' ', text, flags=re.I)
    return text

Die nächste Zelle definiert zwei Hilffunktionen, die für die Visualisierung der Daten verwendet werden.

In [3]:
def set_plot_configs(p):
    p.background_fill_color = '#929191'
    p.border_fill_color = '#666666'
    p.background_fill_alpha = 0.45
    p.border_fill_alpha = 0.25
    p.xgrid.grid_line_color = '#afafaf'
    p.ygrid.grid_line_color = '#afafaf'
    p.xgrid.grid_line_alpha = 0.3
    p.ygrid.grid_line_alpha = 0.3
    p.axis.axis_label_text_font = 'Times New Roman'
    p.axis.major_label_text_font = 'Times New Roman'    
    return p

def set_plot_labels(p,x_values,y_values):
    sorted_df = top_answer_users.sort_values(by=y_values, ascending=False)
    source = dict(
        x=sorted_df[x_values][0:10].values,
        y=sorted_df[y_values][0:10].values,
        names=sorted_df["display_name"][0:10].values
    )    
        
    
    labels = LabelSet(
                x='x',
                y='y',
                text='names',
                level='glyph',
                x_offset=2, 
                y_offset=2, 
                text_font_size="7pt",
                source=ColumnDataSource(source), 
                render_mode='canvas')

    p.add_layout(labels)
    return p

# Visualisierung
In dieser Zelle werden die Experten in einer Grafik visualisiert

In [15]:
# top users diagram
circle_size = 12.5
fill_color = '#F48024'

p = figure(title=' ', plot_width=600, plot_height=400)

curdoc().theme = 'light_minimal'
p.circle(top_answer_users['answer_count'], top_answer_users['answer_reputation'], size=circle_size, fill_color=fill_color, line_color='black', line_width=0.5)
p.xaxis.axis_label = 'Anzahl Posts'
p.yaxis.axis_label = 'Anzahl Reputationspunkte'

p = set_plot_configs(p)
p = set_plot_labels(p, 'answer_count', 'answer_reputation')


show(p)

{'x': array([579, 618, 238, 378, 309, 318, 182, 14, 186, 92], dtype=object), 'y': array([31680, 25015, 22335, 20950, 16625, 14080, 9175, 8445, 8265, 7830],
      dtype=object), 'names': array(['desertnaut', 'lejlot', 'Marcin Możejko', 'Maxim', 'Shai', 'today',
       'Vivek Kumar', 'dga', 'Daniel Möller', 'Martin Thoma'],
      dtype=object)}


# Metriken/Analysen
Die nächsten Zellen zeigen weitere Analysen und Metriken, die die vorherige Analyse ergänzen.

# Antworten - Akzeptierte Antworten
Die folgende Zelle zeigt das Verhältnis der Anzahl von Antworten zu der Anzahl der Akzeptierten Antworten

In [17]:
p = figure(title=' ', plot_width=500, plot_height=500)

curdoc().theme = 'light_minimal'
p.circle(top_answer_users['answer_count'], top_answer_users['accepted_answer_count'], size=circle_size, fill_color=fill_color, line_color='black', line_width=0.5)
p.xaxis.axis_label = 'Anzahl Antworten'
p.yaxis.axis_label = 'Anzahl akzeptierte Anwtorten'

p = set_plot_configs(p)
p = set_plot_labels(p, 'answer_count', 'accepted_answer_count')

show(p)

{'x': array([579, 618, 318, 378, 309, 238, 182, 266, 186, 356], dtype=object), 'y': array([458, 363, 250, 244, 195, 159, 129, 108, 105, 96], dtype=object), 'names': array(['desertnaut', 'lejlot', 'today', 'Maxim', 'Shai', 'Marcin Możejko',
       'Vivek Kumar', 'Prune', 'Daniel Möller', 'Has QUIT--Anony-Mousse'],
      dtype=object)}


# Metrik - Antworten/akzeptierte Antworten
Rate an akzeptierten Antworten auf Grundlagen von Gesamtanzahl der Antworten unter Experten

In [55]:
total_answers = 0

for answer_amount in top_answer_users["answer_count"]:
    total_answers += answer_amount

total_acc_answers = 0
for acc_answer_amount in top_answer_users["accepted_answer_count"]:
    if math.isnan(acc_answer_amount):
        continue
    total_acc_answers += acc_answer_amount

acc_answer_answer_ratios = []

for user in top_answer_users.itertuples():
    acc_answer_answer_ratio = user[8]/user[6]
    acc_answer_answer_ratios.append(acc_answer_answer_ratio)

acc_answer_answer_ratios_df = pd.DataFrame(acc_answer_answer_ratios, index=top_answer_users["display_name"])
acc_answer_answer_ratios_df.columns = ["Ratio"]
acc_answer_answer_ratios_df.fillna(0, inplace=True)
acc_answer_answer_ratios_df = acc_answer_answer_ratios_df.sort_values(by=["Ratio"], ascending=False)
pd.options.display.float_format = "{:,.3f}".format

acc_answer_answer_ratio = total_acc_answers/total_answers
print(acc_answer_answer_ratio)
display(acc_answer_answer_ratios_df)


0.5920076573342905


,Ratio
display_name,
desertnaut,0.791
today,0.786
Vivek Kumar,0.709
Marcin Możejko,0.668
mrry,0.646
Maxim,0.646
dga,0.643
Shai,0.631
MaxU,0.588


# Metrik - Reputation pro Antwort

In [56]:
rep_per_answer_list = []
for user in top_answer_users.itertuples():
    rep_per_answer = user[5]/user[6]
    rep_per_answer_list.append(rep_per_answer)
    
rep_per_answer_df = pd.DataFrame(rep_per_answer_list, index=top_answer_users["display_name"])
rep_per_answer_df.columns = ["Reputation/Antwort"]
rep_per_answer_df.fillna(0, inplace=True)
pd.options.display.float_format = "{:,.3f}".format
rep_per_answer_df = rep_per_answer_df.sort_values(by=["Reputation/Antwort"], ascending=False)
display(rep_per_answer_df)

rep_per_answer_df["Reputation/Antwort"].sum()/20


,Reputation/Antwort
display_name,
runhani,"4,840.000"
dga,603.214
Salvador Dali,188.857
stackoverflowuser2010,164.022
Marcin Możejko,93.845
mrry,92.215
Martin Thoma,85.109
MaxU,61.438
Maxim,55.423


332.05791736784363

# Anzahl Tag pro Experte

In [57]:
tags_per_user = []
for tags in top_answer_users["answer_tags"]:
    tags_per_user.append(len(tags))

tags_per_user_df = pd.DataFrame(tags_per_user, index=top_answer_users["display_name"])
tags_per_user_df.columns = ["Anzahl Tags"]
tags_per_user_df.fillna(0, inplace=True)
pd.options.display.float_format = "{:,.3f}".format
display(tags_per_user_df)

,Anzahl Tags
display_name,
desertnaut,190
lejlot,256
Marcin Możejko,89
Maxim,174
Shai,83
today,102
Vivek Kumar,99
dga,15
Daniel Möller,67


In [ ]:
answer_tags = top_answer_users["answer_tags"]
for tag in answer_tags:
    print(tag)

In [60]:
top_users = sorted_user_df[0:20]
sorted_user_tag_df = pd.DataFrame(top_users, columns=["answer_tags"], index=top_users.index)
sorted_user_tag_df["user"] = top_users.index
sorted_user_tag_df.fillna("0", inplace=True)

# Metrik Reputation in "machine-learning"/Gesamtreputation

In [62]:
top_users_total_rep = top_users_posts.drop_duplicates(subset=["AnswerUserId"])
top_users_total_rep
expert_reputations = []
for expert in top_users.itertuples():
    
    user_reputation = top_users_total_rep.loc[top_users_total_rep['AnswerUserId'] == expert[0]]["AnswerUserReputation"].values[0]          
    display_name = expert[1]
    calculated_rep = expert[5]                    
    #expert_reputations.append([display_name, user_reputation, calculated_rep])
    expert_reputations.append(calculated_rep/user_reputation)
    
top_users = sorted_user_df[0:20]
expert_reputations_df = pd.DataFrame(expert_reputations, index=top_users["display_name"])
expert_reputations_df.columns = ["Ratio"]
expert_reputations_df

,Ratio
display_name,
desertnaut,0.695
lejlot,0.441
Marcin Możejko,0.641
Maxim,0.439
Shai,0.180
today,0.524
Vivek Kumar,0.317
dga,0.407
Daniel Möller,0.112


# Tag Similarity

In [142]:
user_tag_list = []
for i,tags in enumerate(sorted_user_tag_df["answer_tags"]):
        
    if tags != "0":    
        for tag in tags:        
            user_tag = [sorted_user_tag_df.iloc[i]["user"], tag]
            user_tag_list.append(user_tag)
        


user_tag_list_df = pd.DataFrame(user_tag_list, columns=["userid", "tag"])
user_tag_list_df["count"] = 1

user_tags_matrix = user_tag_list_df.pivot(index="userid", columns="tag", values="count")
user_tags_matrix.fillna(0, inplace=True)
user_tag_similarity = cosine_similarity(user_tags_matrix)

sim_df = pd.DataFrame(user_tag_similarity, index=user_tags_matrix.index, columns=user_tags_matrix.index)
display(user_tags_matrix)
display(sim_df)

tag,.net,accord.net,activation-function,adaboost,adam,algorithm,amazon-sagemaker,anaconda,analysis,analytics,...,word-embedding,word2vec,xgbclassifier,xgboost,xls,xor,yellowbrick,yelp,yolo,zero-padding
userid,,,,,,,,,,,,,,,,,,,,,
349130,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
562769,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
712995,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,1.000,1.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000
1060350,0.000,1.000,0.000,0.000,0.000,1.000,1.000,0.000,1.000,0.000,...,1.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000
1090562,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1714410,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2097240,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000
2099607,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,...,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2658050,1.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000


userid,349130,562769,712995,1060350,1090562,1714410,2097240,2099607,2658050,3374996,3574081,4561314,4685471,4785185,5025009,5545260,5741205,5974433,6730309,10908375
userid,,,,,,,,,,,,,,,,,,,,
349130,1.000,0.431,0.361,0.233,0.296,0.398,0.443,0.404,0.326,0.296,0.376,0.397,0.312,0.380,0.282,0.292,0.303,0.444,0.113,0.424
562769,0.431,1.000,0.376,0.336,0.322,0.368,0.324,0.377,0.390,0.406,0.347,0.405,0.377,0.422,0.370,0.268,0.376,0.392,0.115,0.388
712995,0.361,0.376,1.000,0.296,0.330,0.349,0.370,0.435,0.403,0.358,0.324,0.317,0.352,0.418,0.279,0.215,0.300,0.410,0.114,0.323
1060350,0.233,0.336,0.296,1.000,0.173,0.202,0.191,0.204,0.416,0.392,0.179,0.296,0.375,0.433,0.362,0.165,0.338,0.263,0.071,0.210
1090562,0.296,0.322,0.330,0.173,1.000,0.363,0.319,0.310,0.294,0.297,0.385,0.329,0.290,0.285,0.289,0.405,0.304,0.351,0.261,0.315
1714410,0.398,0.368,0.349,0.202,0.363,1.000,0.469,0.402,0.322,0.243,0.330,0.311,0.295,0.385,0.261,0.312,0.230,0.384,0.165,0.397
2097240,0.443,0.324,0.370,0.191,0.319,0.469,1.000,0.508,0.305,0.307,0.386,0.346,0.355,0.319,0.203,0.347,0.256,0.505,0.183,0.410
2099607,0.404,0.377,0.435,0.204,0.310,0.402,0.508,1.000,0.322,0.308,0.329,0.347,0.417,0.369,0.270,0.230,0.299,0.462,0.198,0.422
2658050,0.326,0.390,0.403,0.416,0.294,0.322,0.305,0.322,1.000,0.408,0.227,0.346,0.440,0.508,0.349,0.210,0.356,0.431,0.094,0.282


In [145]:
threshold = 0.5

tag_similarity_filtered = sim_df
tag_similarity_filtered.values[tag_similarity_filtered <= threshold] = 0
tag_similarity_filtered

userid,349130,562769,712995,1060350,1090562,1714410,2097240,2099607,2658050,3374996,3574081,4561314,4685471,4785185,5025009,5545260,5741205,5974433,6730309,10908375
userid,,,,,,,,,,,,,,,,,,,,
349130,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
562769,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
712995,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1060350,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1090562,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1714410,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2097240,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.508,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.505,0.000,0.000
2099607,0.000,0.000,0.000,0.000,0.000,0.000,0.508,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2658050,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.508,0.000,0.000,0.000,0.000,0.000,0.000


In [150]:
tag_sim_graph = nx.from_pandas_adjacency(tag_similarity_filtered,  create_using=nx.DiGraph)


user_dict = sorted_user_tag_df["user"].to_dict()
nx.set_node_attributes(tag_sim_graph, user_dict, "user")


alt.data_transformers.disable_max_rows()
chart = nxa.draw_networkx(
    tag_sim_graph,
    node_color= '#F48024',
    edge_color= '#BCBBBB',
    node_tooltip = ["user"],
    width = 3
).properties(
    width=800,
    height=800
).interactive()
chart


alt.LayerChart(...)

In [81]:
sorted_user_tag_df

,answer_tags,user
4685471,"{backpropagation, java, random-seed, computer-vision, lda, pearson-correlation, nearest-neighbor, face-detection, metrics, autoencoder, ios, lightgbm, tfidfvectorizer, mean-square-error, densenet,...",4685471
2658050,"{multiple-regression, poisson, computer-vision, c#, lua, lda, contour, function, nearest-neighbor, metrics, mle, ruby, word-count, markov, algorithm, resampling, pandas, modeling, grid, pca, glmne...",2658050
5974433,"{caffe, backpropagation, keras, image-recognition, computer-science, evaluate, unsupervised-learning, computer-vision, python, one-hot-encoding, conv-neural-network, plot, non-linear-regression, p...",5974433
712995,"{backpropagation, xor, transfer, list, computer-science, linear-discriminant, tensor, computer-vision, snpe, tensorflow-serving, nearest-neighbor, matrix-multiplication, multi-gpu, autoencoder, ml...",712995
1714410,"{caffe, segmentation-fault, backpropagation, keras, image-recognition, autograd, generative-adversarial-network, protocol-buffers, matlab, unsupervised-learning, computer-vision, tensor, python, l...",1714410
2099607,"{backpropagation, keras, face-recognition, generative-adversarial-network, unsupervised-learning, tensor, computer-vision, python, conv-neural-network, hyperas, ocr, signal-processing, progress-ba...",2099607
3374996,"{keras, oversampling, unsupervised-learning, computer-vision, categorical-data, python, one-hot-encoding, conv-neural-network, hyperparameters, data-mining, labels, anaconda, data-transform, neare...",3374996
5545260,"{deep-learning, c++, optimization, image-recognition, xor, tensorflow, computer-vision, skflow, python, object-detection, neural-network, numpy, conv-neural-network, image-processing, python-2.7}",5545260
2097240,"{backpropagation, keras, zero-padding, generative-adversarial-network, tensor, computer-vision, python, one-hot-encoding, conv-neural-network, unet, training-data, metrics, python-3.x, autoencoder...",2097240
562769,"{caffe, keras, image-recognition, face-recognition, threshold, matlab, computer-vision, python, one-hot-encoding, text-analysis, conv-neural-network, ocr, training-data, non-linear-regression, fea...",562769


# Same tags in all taglists

In [152]:
intersect_set = sorted_user_tag_df.iloc[0]["answer_tags"]
#print(intersect_set)
for i,tags in enumerate(sorted_user_tag_df["answer_tags"][0:19]):
    intersect_set = intersect_set.intersection(tags)
intersect_set

{'deep-learning', 'neural-network', 'python'}

# Count tags

In [ ]:
tag_counter = Counter()
for i,tags in enumerate(sorted_user_tag_df["answer_tags"][0:20]):
    for tag in tags:
        tag_counter[tag] += 1

tag_counter = Counter({k: c for k,c in sorted(tag_counter.items(), key=lambda item: item[1], reverse=True)})

tag_counter

# Gesamtanzahl der Tags

In [76]:
len(tag_counter)

689

# Textähnlichkeiten der Experten

In [ ]:
doc_dict = {}

for i in range(0, len(top_users_posts)):        
    doc_dict[top_users_posts.iloc[i]["AnswerUserId"]] = doc_dict.get(top_users_posts.iloc[i]["AnswerUserId"], '') + ' ' + clean_bodys(top_users_posts.iloc[i]["AnswerBody"])


In [73]:
doc_df = pd.DataFrame.from_dict(doc_dict, orient="index", columns=["Doc"])

,Doc
1060350,"K2 x,z can be 0.Then this value is not well-..."
2658050,"In short yes, you should include each class. ..."
4561314,Here are the original input variables:A is a ...
1714410,If you got from git you should find in fold...
562769,Value iteration is used when you have transit...
4685471,"In documentation, it is mentioned: What you ..."
5025009,I see 3 possible ways to solve this:1 try to...
5974433,To understand how backpropagation is even pos...
4785185,You have already split on weather and gender....
1090562,Majority of machine learning algorithms work ...


In [ ]:
i = 0
col = 0

doc_sim_matrix = []

while i < len(doc_df) - 1:
    print(i)
    doc1 = nlp(doc_df.iloc[i]["Doc"])
    doc_sim_vec = []
    
    j = col + 1
    while j < len(doc_df):
        print(j)
        doc2 = nlp(doc_df.iloc[j]["Doc"])
        sim = doc1.similarity(doc2)
        doc_sim_vec.append(sim)                
        j += 1
    doc_sim_matrix.append(doc_sim_vec)
    col += 1
    i += 1
    print('---')
doc_sim_matrix

In [90]:
pd.DataFrame(doc_sim_matrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.998271,0.992634,0.993036,0.996990,0.994728,0.989189,0.993858,0.996933,0.995932,0.994589,0.989918,0.996549,0.994795,0.995843,0.985585,0.995958,0.993855,0.766346,0.991891
1,0.994438,0.993793,0.996806,0.996435,0.989719,0.995312,0.997449,0.995539,0.994862,0.991263,0.997294,0.994282,0.996877,0.986199,0.996003,0.993757,0.769810,0.993845,NaN
2,0.997096,0.994884,0.993524,0.994794,0.994390,0.996388,0.991792,0.992760,0.996216,0.996506,0.993954,0.997076,0.988790,0.994016,0.994840,0.779966,0.997224,NaN,NaN
3,0.995506,0.992427,0.994934,0.995550,0.995713,0.993450,0.993611,0.996629,0.995825,0.996234,0.997647,0.991585,0.996316,0.996049,0.776746,0.997984,NaN,NaN,NaN
4,0.995417,0.994614,0.996582,0.998064,0.997190,0.995996,0.991776,0.997432,0.997465,0.996675,0.990040,0.998123,0.996326,0.773945,0.994609,NaN,NaN,NaN,NaN
5,0.991593,0.993703,0.997437,0.993970,0.995750,0.991873,0.997488,0.995015,0.995732,0.985639,0.995633,0.995586,0.757010,0.994193,NaN,NaN,NaN,NaN,NaN
6,0.993106,0.993233,0.992416,0.990921,0.993404,0.992842,0.995177,0.993195,0.995065,0.993865,0.996795,0.779525,0.994580,NaN,NaN,NaN,NaN,NaN,NaN
7,0.995966,0.995090,0.994158,0.993664,0.995988,0.994750,0.995447,0.988966,0.995094,0.995113,0.753082,0.995524,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.995245,0.996245,0.993385,0.998221,0.996658,0.997135,0.987684,0.997485,0.996049,0.768711,0.995350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.995769,0.991023,0.996259,0.997098,0.995174,0.988153,0.996704,0.995996,0.760570,0.992043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Schnittmenge der Tags pro Experte

In [191]:
def clean_tags_with_ml(tags):    
    tag_list = []
    # clean tags from '>' and '<' occurences
    tags = re.sub('><', ' ', tags) 
    tags = re.sub('<|>', '', tags)
    # add single tag of tags and add it to lists and sets
    for tag in tags.split():    
        tag_list.append(tag)
    return tag_list

for post in top_users_posts.drop_duplicates(subset="AnswerUserId").itertuples():
    
    user_tag_dict[post[13]] = set(clean_tags_with_ml(post[4]))

for post in top_users_posts.itertuples():
    
    cleaned_tags = clean_tags_with_ml(post[4])
    if len(cleaned_tags) >= 0:
        user_tag_dict[post[13]] = set(user_tag_dict.get(post[13], set()).intersection(cleaned_tags))
user_tag_dict

{'Has QUIT--Anony-Mousse': {'machine-learning'},
 'lejlot': {'machine-learning'},
 'stackoverflowuser2010': {'machine-learning'},
 'Shai': {'machine-learning'},
 'Martin Thoma': {'machine-learning'},
 'desertnaut': set(),
 'seralouk': {'machine-learning'},
 'Marcin Możejko': {'machine-learning'},
 'Prune': {'machine-learning'},
 'Salvador Dali': {'machine-learning'},
 'dga': {'machine-learning', 'tensorflow'},
 'mrry': {'machine-learning'},
 'Maxim': set(),
 'Nicolas Gervais': {'machine-learning'},
 'Dr. Snoopy': {'machine-learning'},
 'MaxU': {'machine-learning'},
 'Daniel Möller': {'machine-learning'},
 'Vivek Kumar': {'machine-learning'},
 'runhani': {'conv-neural-network',
  'convolution',
  'deep-learning',
  'machine-learning',
  'signal-processing'},
 'today': {'machine-learning'}}